In [1]:

import os;
import json;
import builtins
import duckdb
import pandas as pd;
import matplotlib.pyplot as plt
import numpy as np

def get_dataset_from_testcase(run):
    return run[6:-2]

dir = '/home/chesetti/Repos/learned_merge_cleanup/sponge/pgm_vs_flat'
runs = []
index_stats = []
for test_case in os.listdir(dir):
    if test_case == 'build':
        continue
    for run_name in os.listdir(os.path.join(dir, test_case, 'outputs', 'results', 'run')):
        json_file = builtins.open(os.path.join(dir, test_case, 'outputs', 'results', 'run', run_name))
        run = (json.load(json_file))
        run['dataset'] = get_dataset_from_testcase(test_case)
        runs.append(run)
        runs.append(run)
df = pd.json_normalize(runs)
print(df.columns)

Index(['command', 'dataset', 'result.checksum', 'result.duration_ns',
       'result.duration_sec', 'result.inner_disk_fetch',
       'result.inner_disk_fetch_size', 'result.inner_total_bytes_fetched',
       'result.outer_disk_fetch', 'result.outer_disk_fetch_size',
       'result.outer_total_bytes_fetched', 'spec.algo', 'spec.algo_name',
       'spec.check_checksum', 'spec.common_key', 'spec.index.search',
       'spec.index.type', 'spec.inner_table', 'spec.key_size', 'spec.key_type',
       'spec.load_sstable_in_mem', 'spec.name', 'spec.num_threads',
       'spec.outer_table', 'spec.result_path', 'spec.value_size',
       'spec.write_result_to_disk'],
      dtype='object')


In [2]:
epsilon = {
    "pgm256": 256,
    "pgm1024": 1024,
    "pgm2048": 2048,
    "flatpgm256": 256,
    "flatpgm1024": 1024,
    "flatpgm2048": 2048,
    "flatpgm4096": 4096,
    "flatpgm8192": 8192,
    "hashjoin": 0,
    "btree": 0,
    "sj": 0,
    "sj2": 0,
    "standard_merge": 0
};
def indexType(x):
    if "flatpgm" in x:
        return "FLAT_PGM"
    elif "pgm" in x:
        return "PGM"
    elif "btree" in x:
        return "BTREE"
    return "NA"


df["threads"] = df["spec.num_threads"]
df["duration_sec"] = df["result.duration_ns"] / (1000000000)
df["ratio"] = df["spec.common_key"]
df["algo"] = df["spec.algo_name"]
df["join_algo"] = df["spec.algo"]
df["inner_bytes_fetched"] = df["result.inner_total_bytes_fetched"]
df["index_type"] = df["spec.algo_name"].map(lambda x: indexType(x))
df['epsilon'] = df['spec.index.type'].map(lambda x: epsilon[x])

print(df['join_algo'].unique())


['inlj' 'lsj']


In [3]:
def plot_for_dataset(dataset, join_algo):
    rows = duckdb.sql(
        " SELECT ratio, index_type, epsilon, MEDIAN(duration_sec) as v FROM df"
        f" WHERE dataset='{dataset}' AND join_algo='{join_algo}'"
        " GROUP BY ratio, join_algo, index_type, epsilon"
        " ORDER BY ratio"
    ).df()
    rows = rows.pivot(index=['ratio', 'epsilon'], values='v', columns=['index_type'])
    return rows


# INLJ (Raw Lookup)

In [4]:
for dataset in ['fb', 'wiki', 'books', 'osm', 'uniform_dense', 'uniform_sparse', 'normal', 'lognormal']:
    display(dataset, plot_for_dataset(dataset, 'inlj'))

'fb'

index_type      FLAT_PGM        PGM
ratio epsilon                      
1     256      34.156117  39.469889
      2048     35.249738  38.958030
10    256       4.237749   4.799960
      2048      4.855206   5.019735
100   256       0.977560   1.025912
      2048      1.237342   1.264092
1000  256       0.268419   0.274501
      2048      0.465985   0.468841

'wiki'

index_type      FLAT_PGM        PGM
ratio epsilon                      
1     256      13.716520  15.175388
      2048     14.935869  16.523325
10    256       3.065846   3.381930
      2048      3.718721   3.808363
100   256       0.683415   0.725810
      2048      0.875317   0.917314
1000  256       0.225640   0.210115
      2048      0.327308   0.334551

'books'

...

index_type      FLAT_PGM        PGM
ratio epsilon                      
1     256      31.109288  35.031973
      2048     33.084781  35.640530
10    256       4.154219   4.209283
      2048      4.292798   4.421939
100   256       0.925096   0.955879
      2048      1.198191   1.217230
1000  256       0.276882   0.265202
      2048      0.482471   0.470557

'normal'

index_type      FLAT_PGM        PGM
ratio epsilon                      
1     256      26.387389  29.383485
      2048     29.075093  33.359191
10    256       3.537708   3.899677
      2048      4.243542   5.178971
100   256       0.908488   0.939857
      2048      1.249539   1.304167
1000  256       0.274943   0.268433
      2048      0.468021   0.490918

'lognormal'

index_type      FLAT_PGM        PGM
ratio epsilon                      
1     256      25.650273  31.691376
      2048     29.463036  33.798506
10    256       3.365123   4.212493
      2048      4.585393   5.627024
100   256       0.935811   1.622649
      2048      1.464235   1.495417
1000  256       0.285238   0.290718
      2048      0.522310   0.522746

# Learned Sort Join


In [5]:
for dataset in ['fb', 'wiki', 'books', 'osm', 'uniform_dense', 'uniform_sparse', 'normal', 'lognormal']:
    display(dataset, plot_for_dataset(dataset, 'lsj'))

'fb'

index_type      FLAT_PGM        PGM
ratio epsilon                      
1     256      24.993924  31.076291
      2048     28.333396  31.613697
10    256       3.725301   4.459557
      2048      4.197650   4.285085
100   256       0.841785   0.920754
      2048      0.837375   0.883383
1000  256       0.274187   0.290587
      2048      0.411975   0.414342

'wiki'

index_type      FLAT_PGM        PGM
ratio epsilon                      
1     256      11.629625  13.256304
      2048     12.938003  14.460887
10    256       3.032764   3.271661
      2048      3.292362   3.685433
100   256       0.599972   0.630771
      2048      0.603141   0.645177
1000  256       0.204984   0.204099
      2048      0.241807   0.245447

'books'

...

index_type      FLAT_PGM        PGM
ratio epsilon                      
1     256      25.246547  28.984153
      2048     28.491774  29.710565
10    256       3.597230   3.835992
      2048      3.890131   4.136220
100   256       0.821902   0.899803
      2048      0.833240   0.835220
1000  256       0.257811   0.283472
      2048      0.442412   0.434035

'normal'

index_type      FLAT_PGM        PGM
ratio epsilon                      
1     256      24.686158  28.287737
      2048     27.834476  31.221273
10    256       3.396363   4.066837
      2048      3.949190   4.163381
100   256       0.816836   0.853736
      2048      0.814305   0.869927
1000  256       0.270962   0.263467
      2048      0.432573   0.433598

'lognormal'

index_type      FLAT_PGM        PGM
ratio epsilon                      
1     256      23.965234  29.991976
      2048     26.967841  30.476574
10    256       3.372891   4.041178
      2048      3.643738   4.344789
100   256       0.832283   0.888538
      2048      1.262608   0.866543
1000  256       0.345975   0.291441
      2048      0.416492   0.439766